In [1]:
%load_ext autoreload
%autoreload 2
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament,
    Match
)
import sqlite3
import pandas as pd
import uuid
import requests
import json

In [2]:
str(uuid.uuid4())

'a57d7a6d-13fb-4947-a161-c34664a83a49'

In [3]:
db.session.close()

In [4]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [5]:
# purge all tables first

for Model in [BracketModel, TournamentModel, MatchModel, RoundModel, UserModel]:
    purge(Model)
db.session.commit()

In [6]:
# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)

# print empty user table
df

,id,username,email,password_hash,about_me,last_seen


In [7]:
# register user like
# curl -i -X POST -H "Content-Type: application/json" -d '{"username":"miguel","password":"python"}' http://127.0.0.1:5000/api/users

In [8]:
url = 'http://127.0.0.1:5000/api/users'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "username":"miguel",
    "password":"python",
    "email":"miguel@gmail.com"
})

r = requests.post(url, data=payload, headers=headers)

In [9]:
r.headers['location']

'http://127.0.0.1:5000/api/user/1'

In [10]:
vars(r)

{'_content': b'{"username":"miguel"}\n',
 '_content_consumed': True,
 '_next': None,
 'status_code': 201,
 'headers': {'Content-Type': 'application/json', 'Content-Length': '22', 'location': 'http://127.0.0.1:5000/api/user/1', 'Server': 'Werkzeug/1.0.0 Python/3.6.9', 'Date': 'Tue, 25 Feb 2020 00:14:56 GMT'},
 'raw': <urllib3.response.HTTPResponse at 0x7f6d4e44c588>,
 'url': 'http://127.0.0.1:5000/api/users',
 'encoding': None,
 'history': [],
 'reason': 'CREATED',
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(0, 0, 96290),
 'request': <PreparedRequest [POST]>,
 'connection': <requests.adapters.HTTPAdapter at 0x7f6d4e432b00>}

In [11]:
payload

'{"username": "miguel", "password": "python", "email": "miguel@gmail.com"}'

In [12]:
# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,username,email,password_hash,about_me,last_seen
0,1,miguel,miguel@gmail.com,pbkdf2:sha256:150000$D59ZAdor$a1a8a5f31f1c7fa2...,None,2020-02-25 00:14:56.787091


In [13]:
# post the rest of the users
users = [
    ('TPN', 'TPN@example.com'),
    ('AngryFalco', 'AngryFalco@example.com'),
    ('Sunrisebanana', 'Sunrisebanana@example.com'),
    ('Ptolemy', 'Ptolemy@example.com'),
    ('Vik', 'Vik@example.com'),
    ('Kevin', 'Kevin@example.com'),
    ('Spaceghost', 'Spaceghost@example.com'),
    ('Burnaby', 'Burnaby@example.com'),
]

for username, email in users:
    payload = json.dumps({
        "username":username,
        "password":username.lower(),
        "email":email.lower()
    })

    r = requests.post(url, data=payload, headers=headers)
#     print(vars(r))

In [14]:
# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,username,email,password_hash,about_me,last_seen
0,1,miguel,miguel@gmail.com,pbkdf2:sha256:150000$D59ZAdor$a1a8a5f31f1c7fa2...,None,2020-02-25 00:14:56.787091
1,2,TPN,tpn@example.com,pbkdf2:sha256:150000$fjdasM84$04877a70ead66657...,None,2020-02-25 00:17:57.703017
2,3,AngryFalco,angryfalco@example.com,pbkdf2:sha256:150000$hOmSPHZO$1b44835d4066698f...,None,2020-02-25 00:17:57.787550
3,4,Sunrisebanana,sunrisebanana@example.com,pbkdf2:sha256:150000$JlDVOUJk$820419864581a2f6...,None,2020-02-25 00:17:57.870550
4,5,Ptolemy,ptolemy@example.com,pbkdf2:sha256:150000$7wbyWWXX$ae14c50a8f696a9d...,None,2020-02-25 00:17:57.953902
5,6,Vik,vik@example.com,pbkdf2:sha256:150000$N8uXgS95$eab571193718947f...,None,2020-02-25 00:17:58.034356
6,7,Kevin,kevin@example.com,pbkdf2:sha256:150000$SPKH7uqs$6a98f94b437b6e9a...,None,2020-02-25 00:17:58.116027
7,8,Spaceghost,spaceghost@example.com,pbkdf2:sha256:150000$NCnWIeMr$ce1df291bf0ec094...,None,2020-02-25 00:17:58.197036
8,9,Burnaby,burnaby@example.com,pbkdf2:sha256:150000$SaPrYBbL$fd2a93abf2bd4cc5...,None,2020-02-25 00:17:58.277980


In [16]:
r.headers['location']

'http://127.0.0.1:5000/api/user/9'

In [7]:
# # add all users to users table
# users = [
#     UserModel(username='TPN', email='TPN@example.com'.lower()), 
#     UserModel(username='AngryFalco', email='AngryFalco@example.com'.lower()), 
#     UserModel(username='Sunrisebanana', email='Sunrisebanana@example.com'.lower()), 
#     UserModel(username='Ptolemy', email='Ptolemy@example.com'.lower()), 
#     UserModel(username='Vik', email='Vik@example.com'.lower()), 
#     UserModel(username='Kevin', email='Kevin@example.com'.lower()), 
#     UserModel(username='Spaceghost', email='Spaceghost@example.com'.lower()), 
#     UserModel(username='Burnaby', email='Burnaby@example.com'.lower()), 
# ]

In [11]:
users[0].username.lower()

'tpn'

In [7]:
[db.session.add(u) for u in users]
db.session.commit()

In [8]:
# Fall Charity LAN 2018 Melee Singles users
N_COMPETITORS = 8
seeds = [i+1 for i in range(N_COMPETITORS)]
usernames = [
    'TPN',
    'AngryFalco',
    'Sunrisebanana',
    'Ptolemy',
    'Vik',
    'Kevin',
    'Spaceghost',
    'Burnaby',
]
np.random.shuffle(usernames)
np.random.shuffle(seeds)

In [9]:
tuple_list = [(usernames[i], seeds[i]) for i in range(N_COMPETITORS)]

In [10]:
btype = BracketTypes.DOUBLE_ELIMINATION
t = Tournament(tuple_list, btype)

In [11]:
tournament_name = 'Fall Charity LAN 2018 Melee Singles'
TO = t.bracket.entrants[0]

# initial post to db
t.post_to_db(tournament_name, TO,)

In [12]:
# post self references in matches separately
for r in t.bracket.rounds:
    for m in r.matches:
        m.post_self_refs()

In [13]:
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)
df

,id,username,email,password_hash,about_me,last_seen
0,1,TPN,tpn@example.com,None,None,2020-02-20 01:20:58.042862
1,2,AngryFalco,angryfalco@example.com,None,None,2020-02-20 01:20:58.043367
2,3,Sunrisebanana,sunrisebanana@example.com,None,None,2020-02-20 01:20:58.043462
3,4,Ptolemy,ptolemy@example.com,None,None,2020-02-20 01:20:58.043541
4,5,Vik,vik@example.com,None,None,2020-02-20 01:20:58.043612
5,6,Kevin,kevin@example.com,None,None,2020-02-20 01:20:58.043696
6,7,Spaceghost,spaceghost@example.com,None,None,2020-02-20 01:20:58.043772
7,8,Burnaby,burnaby@example.com,None,None,2020-02-20 01:20:58.043859


In [14]:
pd.read_sql_query('select * from tournament', cnx)

,id,n_entrants,name,organizer_id
0,1,8,Fall Charity LAN 2018 Melee Singles,2


In [15]:
pd.read_sql_query('select * from bracket', cnx)

,id,bracket_type,tournament_id
0,1,DOUBLE_ELIMINATION,1


In [16]:
pd.read_sql_query('select * from bracket_entrants', cnx)

,user_id,bracket_id
0,2,1
1,8,1
2,5,1
3,3,1
4,6,1
5,7,1
6,1,1
7,4,1


In [17]:
pd.read_sql_query('select * from round', cnx)

,id,number,winners,bracket_id
0,1,1,1,1
1,2,2,1,1
2,3,3,1,1
3,4,4,1,1
4,5,5,1,1
5,6,1,0,1
6,7,2,0,1
7,8,3,0,1
8,9,4,0,1


In [18]:
df = pd.read_sql_query('select * from match', cnx)
df
# df.loc[df['uuid'] == '94de92a2-13ac-4e9a-9bf2-78636100a7b1']

,id,uuid,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,0f9a1604-97d0-4a42-b510-56f7ac0f5ae6,2.0,None,4.0,None,None,1,5.0,10.0
1,2,f4f7edfb-9d3b-4348-88a6-cf4dc2eb7cf2,8.0,None,1.0,None,None,1,5.0,10.0
2,3,e3f3fe1d-4907-45e0-a70e-57b32aebec1e,5.0,None,7.0,None,None,1,6.0,11.0
3,4,d2785cd6-9819-4005-8404-b3cd4059e164,3.0,None,6.0,None,None,1,6.0,11.0
4,5,f31792a1-2cc5-4b81-9771-3f0d539c0734,NaN,None,NaN,None,None,2,7.0,12.0
5,6,f8cf1b92-7bb5-4a74-b88d-7e9dd78217f5,NaN,None,NaN,None,None,2,7.0,13.0
6,7,ec8fb7c6-f4bd-4366-a720-d1470479b824,NaN,None,NaN,None,None,3,8.0,15.0
7,8,80667cdb-737f-45e4-ad90-e9cf472506df,NaN,None,NaN,None,None,4,NaN,NaN
8,9,e2e48a25-f7e9-4ada-9dc0-b6a1e889aa36,NaN,None,NaN,None,None,5,NaN,NaN
9,10,e3f465e6-eb1a-4e62-9537-dfa1aaf32132,NaN,None,NaN,None,None,6,12.0,NaN
